In [2]:
#Warning
import warnings
warnings.filterwarnings("ignore")

#Main
import os
import glob
import cv2
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
import  numpy   as np
import pandas as pd
import gc
import string
import time
import time
import random
import imutils
from PIL import Image
from tqdm import tqdm
tqdm.pandas()

#Visualisation
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE

#Model
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
#Configuration
class CFG: 
    batch_size = 64
    img_height = 64
    img_width = 64
    epochs = 10
    num_classes = 29
    img_channels =3
    
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"]  = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    

In [6]:
#Labels
TRAIN_PATH = "asl_alphabet_train/asl_alphabet_train/"
labels = []
alphabet = list(string.ascii_uppercase)
labels.extend(alphabet)
labels.extend(["del","nothing","space"])
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


## Data PreProcessing
This code creates metadata for the ASL alphabet dataset by getting all the image files for each label, creating a list of image paths, and a corresponding list of labels. It then creates a Pandas dataframe using these two lists with columns "image_path" and "label".

In [7]:
#Create Metadata

list_path = []
list_labels = []

for label in labels:
    label_path = os.path.join(TRAIN_PATH, label, "*")
    image_files = glob.glob(label_path)
    
    sign_label=[label]* len(image_files)
    
    list_path.extend(image_files)
    list_labels.extend(sign_label)
    
metadata=pd.DataFrame({
    
    "image_path":list_path,
    "label":list_labels    
})

metadata

,image_path,label
0,asl_alphabet_train/asl_alphabet_train/A/A402.jpg,A
1,asl_alphabet_train/asl_alphabet_train/A/A218.jpg,A
2,asl_alphabet_train/asl_alphabet_train/A/A1790.jpg,A
3,asl_alphabet_train/asl_alphabet_train/A/A2874.jpg,A
4,asl_alphabet_train/asl_alphabet_train/A/A2443.jpg,A
...,...,...
28294,asl_alphabet_train/asl_alphabet_train/J/J1183.jpg,J
28295,asl_alphabet_train/asl_alphabet_train/J/J2094.jpg,J
28296,asl_alphabet_train/asl_alphabet_train/J/J1938.jpg,J
28297,asl_alphabet_train/asl_alphabet_train/J/J1947.jpg,J


In [8]:
#Split Dataset to train 0.7, Val 0.15 and Test 0.15
X_train, X_test, y_train, y_test = train_test_split(
    metadata["image_path"],metadata["label"],
    test_size=0.15,
    random_state=2023,shuffle=True,
    stratify=metadata["label"]
)

data_train = pd.DataFrame({
    "image_path": X_train,
    "label": y_train
})

X_train, X_val, y_train, y_val = train_test_split(
    data_train["image_path"], data_train["label"],
    test_size=0.15/0.70,
    random_state=2023,
    shuffle=True,
    stratify=data_train["label"]
)

data_train = pd.DataFrame({
    "image_path": X_train,
    "label": y_train
})

data_val = pd.DataFrame({
    "image_path": X_val,
    "label": y_val
})

data_test = pd.DataFrame({
    "image_path": X_test,
    "label": y_test
})

display(data_train)
display(data_val)
display(data_test)

,image_path,label
26089,asl_alphabet_train/asl_alphabet_train/I/I2538.jpg,I
11727,asl_alphabet_train/asl_alphabet_train/D/D350.jpg,D
8217,asl_alphabet_train/asl_alphabet_train/C/C2345.jpg,C
10876,asl_alphabet_train/asl_alphabet_train/D/D428.jpg,D
14578,asl_alphabet_train/asl_alphabet_train/E/E2021.jpg,E
...,...,...
26288,asl_alphabet_train/asl_alphabet_train/I/I737.jpg,I
21396,asl_alphabet_train/asl_alphabet_train/H/H2019.jpg,H
8752,asl_alphabet_train/asl_alphabet_train/C/C1543.jpg,C
14518,asl_alphabet_train/asl_alphabet_train/E/E2707.jpg,E


,image_path,label
5837,asl_alphabet_train/asl_alphabet_train/B/B620.jpg,B
27738,asl_alphabet_train/asl_alphabet_train/J/J1703.jpg,J
18774,asl_alphabet_train/asl_alphabet_train/G/G2118.jpg,G
27264,asl_alphabet_train/asl_alphabet_train/J/J1146.jpg,J
16749,asl_alphabet_train/asl_alphabet_train/F/F1409.jpg,F
...,...,...
2227,asl_alphabet_train/asl_alphabet_train/A/A2161.jpg,A
7614,asl_alphabet_train/asl_alphabet_train/C/C1426.jpg,C
10949,asl_alphabet_train/asl_alphabet_train/D/D2436.jpg,D
21289,asl_alphabet_train/asl_alphabet_train/H/H1839.jpg,H


,image_path,label
20644,asl_alphabet_train/asl_alphabet_train/G/G1048.jpg,G
12827,asl_alphabet_train/asl_alphabet_train/E/E37.jpg,E
3588,asl_alphabet_train/asl_alphabet_train/B/B1711.jpg,B
2332,asl_alphabet_train/asl_alphabet_train/A/A1451.jpg,A
27371,asl_alphabet_train/asl_alphabet_train/J/J1433.jpg,J
...,...,...
13930,asl_alphabet_train/asl_alphabet_train/E/E2282.jpg,E
26676,asl_alphabet_train/asl_alphabet_train/I/I75.jpg,I
4555,asl_alphabet_train/asl_alphabet_train/B/B547.jpg,B
4847,asl_alphabet_train/asl_alphabet_train/B/B1916.jpg,B


## Data Augmentation
This code defines a function named data_augmentation() which generates image data generators using the ImageDataGenerator class from the Keras library. The function defines three generators for training, validation, and testing datasets respectively using the flow_from_dataframe() method of the ImageDataGenerator class. The generators take the image path and label information from dataframes data_train, data_val, and data_test and apply the same data augmentation technique of rescaling the image pixel values to a range of 0 to 1. The batch size, image size, and number of classes are also defined using the CFG class attributes.

In [9]:
#Data Augmentation (Just Rescale)
def data_augmentation():
    datagen = ImageDataGenerator(rescale=1/255.,)
    #Training Dataset
    train_generator = datagen.flow_from_dataframe(
        data_train,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height,CFG.img_width)      
    )
    
    #Validation Dataset
    validtion_generator = datagen.flow_from_dataframe(
        data_val,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height,CFG.img_width)  
        
    )
    
    # Validation Dataset
    validation_generator = datagen.flow_from_dataframe(
        data_val,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width),
    )
    
    # Testing Dataset
    test_generator = datagen.flow_from_dataframe(
        data_test,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=1,
        target_size=(CFG.img_height, CFG.img_width),
        shuffle=False
    )
    
    return train_generator, validation_generator, test_generator

In [10]:
seed_everything(2023)
train_generator, validation_generator, test_generator = data_augmentation()

Found 18899 validated image filenames belonging to 10 classes.
Found 5155 validated image filenames belonging to 10 classes.
Found 5155 validated image filenames belonging to 10 classes.
Found 4245 validated image filenames belonging to 10 classes.


# Model Building

In [11]:


# Load VGG16 model and modify for ASL recognition
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(CFG.img_height, CFG.img_width, CFG.img_channels))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(29, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

display(model.summary())
display(tf.keras.utils.plot_model(model, to_file='vgg16.png', show_shapes=True))




: 

: 

## Training

In [53]:
# Compile and train the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('asl_vgg16_best_weights.h5', save_best_only=True, monitor='val_accuracy', mode='max')

In [54]:
# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // CFG.batch_size,
    epochs=CFG.epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // CFG.batch_size,
    callbacks=[checkpoint]
)

Epoch 1/10


2023-07-01 15:20:17.841493: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-01 15:20:18.447867: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: logits and labels must be broadcastable: logits_size=[64,29] labels_size=[64,10]
	 [[{{node categorical_crossentropy/softmax_cross_entropy_with_logits}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/home/gesskay/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/gesskay/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/gesskay/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/gesskay/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/gesskay/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/gesskay/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/ge

In [ ]:
model.save('Trained_Model.h5')

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('Trained_Model.h5')

# Show the model architecture
new_model.summary()

# Model Evaluation

## Model Testing


In [ ]:
scores = model.evaluate(test_generator)
print("%s: %.2f%%" % ("Evaluate Test Accuracy", scores[1]*100))

## Training Loss and Metrics


In [ ]:
# Visualize Training and Validation Results

# Create Subplot
fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=["Model Loss", "Model Accuracy"], 
)

# Configuration Plot
class PlotCFG:
    marker_size = 8
    line_size = 2
    train_color = "#76503d"
    valid_color = "#deb392"

# Loss Plot
loss = history.history['loss']
val_loss = history.history['val_loss']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(loss)+1), y=loss,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Loss"
    ), row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_loss)+1), y=val_loss,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.valid_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.valid_color, width=PlotCFG.line_size),
        name="Validation Loss"
    ), row=1, col=1
)

# Accuracy Plot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(acc)+1), y=acc,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Accuracy"
    ), row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_acc)+1), y=val_acc,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.valid_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.valid_color, width=PlotCFG.line_size),
        name="Validation Accuracy"
    ), row=1, col=2
)

# Update Axes
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=2)
fig.update_yaxes(title="Categorical Loss", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_yaxes(title="Accuracy", linecolor="Black", ticks="outside", row=1, col=2)

# Update Layout
fig.update_layout(
    title="Training Loss and Metrics", title_x=0.5,
    width=950, height=400,
    showlegend=False,
    plot_bgcolor="White",
    paper_bgcolor="White"
)

# Show
fig.show(iframe_connected=True)

## Confusion Matrix


In [ ]:
# Confusion Matrix
fine_tuned_model = load_model("/kaggle/working/asl_vgg16_best_weights.h5")
predictions = fine_tuned_model.predict(test_generator)

# Get the true labels from the generator
true_labels = test_generator.classes

# Compute the confusion matrix using tf.math.confusion_matrix
confusion_matrix = tf.math.confusion_matrix(
        labels=true_labels,
        predictions=predictions.argmax(axis=1),
        num_classes=29)

In [ ]:
# Create Figure
fig = go.Figure()

# Heatmap
fig.add_trace(
    go.Heatmap(
        z=confusion_matrix,
        x=labels,
        y=labels,
        text=confusion_matrix,
        texttemplate="<b>%{text}</b>",
        textfont={"size":8},
        colorscale=[[0, '#f4f4f4'],[1.0, '#76503d']],
        showscale = False,
        ygap = 5,
        xgap = 5,
        hovertemplate=
        '''
        Actual: %{y}<br>
        Predicted: %{x}<br>
        Total: %{text}
        ''',
        name="Confusion Matrix"
    )
)

# Update Axes
fig.update_xaxes(title="<b>Predicted Values</b>", tickfont_size=10)
fig.update_yaxes(title="<b>Actual Values</b>", tickfont_size=10)

# Update Layout
fig.update_layout(title_text='Confusion Matrix', title_x=0.5, font_size=14,
                  width=1050, 
                  height=1115,
                  plot_bgcolor='white',
                  showlegend=False,
)

# Show
fig.show()